##### Finite State Machine
5-element tuple: $(Q, \Sigma, \delta, q_{0}, F)$
- Q: finite set of states
- $\Sigma$: input symbols
- $\delta$: transition function ($\delta$: Q x $\Sigma$ $\rightarrow$ Q)
- $q_{0}$: starting state
- $F$: final state

##### Generic Framework

In [91]:
from typing import Any, List, Optional
from abc import abstractmethod
from datetime import datetime
import pickle

class State():
    def __init__(self, id: str, val: Optional[Any]=None) -> None:
        self.id = id
        self.value = val
    @property
    def value(self):
        return self._val
    
    @value.setter
    def value(self, val: Any):
        self._val = val
    
    @abstractmethod
    def reset_state(self): #to reset the value
        pass
    @abstractmethod
    def update_state(self): #to update the value
        pass

    def __str__(self):
        return f"State: {self.id}, val: {self.value}"

class FSM():
    def __init__(self, 
                 states: List[State], #Q
                 start_state: State, #q0
                 end_state: State, #F
                 symbols: List[str], #Sigma
                 current_state: Optional[State]=None, #for persistence
                 persist_on_failure: Optional[bool]=False
                 ) -> None:
        self.states = states
        self.state_lookup = {state.id: state for state in self.states}
        self.start_state = start_state
        self.end_state = end_state
        self.symbols = symbols
        self.persist_on_failure = persist_on_failure
        self.current_state = self.start_state if not current_state else current_state

    def _process_symbol(self, inp_symbol: str) -> State:
        next_state = self._transition_fn(self.current_state, inp_symbol)
        return next_state
    
    def process_symbols(self, seq: List[str]) -> State:
        try:
            for symbol in seq:
                log = f"({self.current_state.id}) --\'{symbol}\'--> "
                print(log, end="")
                self.current_state = self._process_symbol(symbol)
                if self.current_state==self.end_state:
                    print(f"({self.current_state.id})")
                    return self.current_state
        except Exception as e:
            print("Encountered a failure!")
            print("Details:-", e)
            if self.persist_on_failure:
                print("Persisting FSM state..")
                self.persist_state()
                return 
        print(f"({self.current_state.id})")
        return self.current_state

    def persist_state(self):
        file_name = f"fsm_obj_{datetime.now().strftime('%Y_%m_%d')}.pkl"
        print(f"States persisted in file: {file_name}")
        pickle.dump(self, open(file_name, "wb"))
    
    #tried to store just the values, it got complicated, considering the challenge to resolve state object types
    def restore_machine(self, pkl_file: str):
        self = pickle.load(open(pkl_file, 'rb'))
        return self

    @abstractmethod
    def _transition_fn(self, inp: State, symbol: str) -> State:
        return
    
    def __str__(self):
        return f"""Possible States: {[str(c) for c in self.states]}\nCurrent State: {str(self.current_state)}"""


##### Design for Toy Problem

##### 2 Conditions to terminate the FSM:-
-You stop it manually in which case, we go to the stop (q_f) state
OR<br>
-You experience failure (in our case, I consider symbol not found as one of the use case for failure), in this case,
I persist the FSM

<img src="design.jpg" alt="Design" style="width: 550px;"/>


In [92]:
class LollipopCounter(State):
    def __init__(self, id: str) -> None:
        super().__init__(id=id, val=0)

    def reset_state(self):
        self.value = 0
    
    def update_state(self):
        self.value +=1
    
class LollipopMachine(FSM):
    def __init__(self, persist_on_failure: Optional[bool]=False) -> None:
        lemon_counter = LollipopCounter(id='Q_l')
        strawberry_couter = LollipopCounter(id='Q_s')
        start = State(id='Q0')
        error = State(id='Q_e')
        end = State(id='Q_f')
        super().__init__(states=[start, 
                                 lemon_counter, 
                                 strawberry_couter, 
                                 error, 
                                 end],
                         symbols=['s', 'l', 'c'],
                         start_state=start, 
                         end_state=end,
                         current_state=start,
                         persist_on_failure=persist_on_failure)
    
    def _transition_fn(self, curr_state: State, symbol: str) -> State:
        if symbol not in self.symbols:
            raise Exception("Invalid Symbol")
        if symbol=='c':
            next_state = self.state_lookup["Q_f"]
            return next_state
        if symbol=='l':
            next_state = self.state_lookup["Q_l"]
        elif symbol=='s':
            next_state = self.state_lookup["Q_s"]
        if curr_state.id!=next_state.id:
            next_state.reset_state()
        next_state.update_state()
        if next_state.value!=0 and next_state.value%3==0:
            next_state.reset_state()
            next_state = self.state_lookup["Q_e"]
        return next_state
        

In [93]:
import random
def generate_symbols(num=20, symbols: List[str]=None):
    if not symbols:
        raise Exception("Provide symbols")
    for _ in range(num):
        yield random.choice(symbols)

In [95]:
machine = LollipopMachine(persist_on_failure=False)
#streaming
# final_state = machine.process_symbols(generate_symbols(num=20,symbols=['l','s']))
#static sequence
final_state = machine.process_symbols(list("slsssssllllsssllls"))
# print(f"{final_state.id}")


(Q0) --'s'--> (Q_s) --'l'--> (Q_l) --'s'--> (Q_s) --'s'--> (Q_s) --'s'--> (Q_e) --'s'--> (Q_s) --'s'--> (Q_s) --'l'--> (Q_l) --'l'--> (Q_l) --'l'--> (Q_e) --'l'--> (Q_l) --'s'--> (Q_s) --'s'--> (Q_s) --'s'--> (Q_e) --'l'--> (Q_l) --'l'--> (Q_l) --'l'--> (Q_e) --'s'--> (Q_s)


In [581]:
l = LollipopMachine()
b = l.restore_machine("fsm_obj_2024_03_03.pkl")
# print("Current", str(b.current_state))